In [35]:
using LinearAlgebra
using Plots
import StatsBase
include("src/Trajectories.jl")
using .QuTaM
using Random

In [31]:
import Distributions

In [45]:
d = Distributions.Exponential(1)
Random.seed!(123)
sample = rand(d, 100)
a = Distributions.fit(Distributions.Exponential, sample).θ

0.8892515029363947

In [43]:
?a

search: a as



No documentation found for private symbol.

`a` is of type `Distributions.Exponential{Float64}`.

# Summary

```
struct Distributions.Exponential{Float64}
```

# Fields

```
θ :: Float64
```

# Supertype Hierarchy

```
Distributions.Exponential{Float64} <: Distributions.Distribution{Distributions.Univariate, Distributions.Continuous} <: Distributions.Sampleable{Distributions.Univariate, Distributions.Continuous} <: Any
```
